In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [30]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [59]:

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
    # random 하게 하나를 예시로 주는 함수.
    def select_examples(self, example_selector):
        from random import choice
        return [choice(self.examples)]
# test 용 example_selector는 사용하고 있지 않기에 아무값이나 넣음
RandomExampleSelector(examples=examples).select_examples(example_selector="Korea")

[{'question': 'What do you know about Greece?',
  'answer': '\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        '}]

In [60]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

# 랜덤한 값 하나를 돌려주는 selector 
example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Korea")


'Human: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Korea?'

In [61]:
chain = prompt | chat
chain.invoke(
    {
        "country":"Korea"
    }
)

AI: 
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi, Bibimbap, Bulgogi
        Currency: South Korean Won

AIMessageChunk(content='AI: \n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi, Bibimbap, Bulgogi\n        Currency: South Korean Won')